# Notebook to remove confounds from a cifti image and apply LPF

There seem to be 2 functions for removing confounds, filtering, etc:

- https://nilearn.github.io/modules/generated/nilearn.image.clean_img.html
- https://nilearn.github.io/modules/generated/nilearn.signal.clean.html

Here I will apply clean_img to the cifti data

In [1]:
import numpy as np
from scipy import signal
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set("paper", "white")

from nilearn.image import load_img
from nilearn.image import clean_img
from nilearn.image import smooth_img
from nilearn.image import index_img
from nilearn.signal import clean
from os import walk

from utility import var_to_nan
from pathlib import Path


import nibabel as nib
from nibabel import cifti2
%matplotlib inline

## Load a sub whole brain data

In [2]:
brainpath='/om2/user/jsmentch/projects/nat_img/sourcedata/data/budapest/brain/sub-sid000007_task-movie_run-1_space-fsLR_den-91k_bold.dtseries.nii'
brainimg = load_img(brainpath) # load func img

confound_path = '/om2/user/jsmentch/projects/nat_img/sourcedata/data/budapest/brain/sub-sid000007_task-movie_run-1_desc-confounds_timeseries.tsv'

data = pd.read_csv(confound_path, sep='\t')
#confounds=data[['global_signal', 'csf', 'white_matter','rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']].to_numpy()
confounds=data[['global_signal', 'csf', 'white_matter', 'framewise_displacement', 'a_comp_cor_00', 'a_comp_cor_01', 'a_comp_cor_02', 'a_comp_cor_03', 'a_comp_cor_04', 'a_comp_cor_05', 'rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']].to_numpy()

func_data=np.array(brainimg.dataobj)
func_data = var_to_nan(func_data,0.5)

#func_cln = clean_img(func,detrend=True,confounds=confounds,low_pass=0.1,t_r=1.5)
func_data_clean = clean(func_data,detrend=True,standardize='zscore',confounds=confounds,low_pass=0.1,t_r=1)
func_cln = nib.Cifti2Image(func_data_clean, brainimg.header)

ValueError: array must not contain infs or NaNs

In [ ]:
#export
func_cln.to_filename(export_path+sub+'_clean_task-MerlinMovie_space-fsLR_den-91k_bold.dtseries.nii')

## Do the same for all budapest subjects

In [3]:
# list all budapest subjects
scratch_fmriprep = '/nobackup/scratch/Mon/jsmentch/nat_img/sourcedata/data/budapest/brain/ds003017/derivatives/fmriprep/'
subject_list = list(walk(scratch_fmriprep))[0][1]
subject_list.remove('logs')
subject_list.remove('derivatives')

In [20]:
#export subject list to a text file
textfile = open("budapest_subjects.txt", "w")
for element in subject_list:
    textfile.write(element + "\n")
textfile.close()

In [20]:
for sub in subject_list:
    for run in np.arange(1,6):
        brainpath=f'{scratch_fmriprep}{sub}/func/{sub}_task-movie_run-{run}_space-fsLR_den-91k_bold.dtseries.nii'
        confound_path = f'{scratch_fmriprep}{sub}/func/{sub}_task-movie_run-{run}_desc-confounds_timeseries.tsv'
        export_path = f'{scratch_fmriprep}derivatives/cleaned/'
        exportfile = Path(f'{export_path}{sub}_run{run}_clean_space-fsLR_den-91k_bold.dtseries.nii')
#         if not exportfile.is_file(): # if it doesn't exist already, run it
        if Path(brainpath).is_file():
            print(f'running {sub}')
            #load confounds
            data = pd.read_csv(confound_path, sep='\t')
            confounds=data[['global_signal', 'csf', 'white_matter', 'framewise_displacement', 'a_comp_cor_00', 'a_comp_cor_01', 'a_comp_cor_02', 'a_comp_cor_03', 'a_comp_cor_04', 'a_comp_cor_05', 'rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']].to_numpy()
            confounds = np.nan_to_num(confounds)
            #confounds=data[['global_signal', 'csf', 'white_matter','rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']].to_numpy()
            #load brain data
            brainimg = load_img(brainpath) # load func img
            func_data=np.array(brainimg.dataobj)
            func_data = var_to_nan(func_data,0.5)
            #clean
            #func_cln = clean_img(func,detrend=True,confounds=confounds,low_pass=0.1,t_r=1.5)
            func_data_clean = clean(func_data,detrend=True,standardize='zscore',confounds=confounds,low_pass=0.1,t_r=1)
            func_cln = nib.Cifti2Image(func_data_clean, brainimg.header)
            #export
            func_cln.to_filename(f'{export_path}{sub}_run{run}_clean_space-fsLR_den-91k_bold.dtseries.nii')
        else:
            print('dne',sub)
#         else:
#             print(f'skipping {sub}, already ran')

running sub-sid000029
running sub-sid000029
running sub-sid000029
running sub-sid000029
running sub-sid000029
running sub-sid000020
running sub-sid000020
running sub-sid000020
running sub-sid000020
running sub-sid000020
running sub-sid000034
running sub-sid000034
running sub-sid000034
running sub-sid000034
running sub-sid000034
running sub-sid000560
running sub-sid000560
running sub-sid000560
running sub-sid000560
running sub-sid000560
running sub-sid000142
running sub-sid000142
running sub-sid000142
running sub-sid000142
running sub-sid000142
running sub-sid000120
running sub-sid000120
running sub-sid000120
running sub-sid000120
running sub-sid000120
running sub-sid000134
running sub-sid000134
running sub-sid000134
running sub-sid000134
running sub-sid000134
running sub-sid000050
running sub-sid000050
running sub-sid000050
running sub-sid000050
running sub-sid000050
running sub-sid000021
running sub-sid000021
running sub-sid000021
running sub-sid000021
running sub-sid000021
running su

In [ ]:
# same as above but skip if already ran
# for sub in subject_list:
#     for run in np.arange(1,6):
#         brainpath=f'{scratch_fmriprep}{sub}/func/{sub}_task-movie_run-{run}_space-fsLR_den-91k_bold.dtseries.nii'
#         confound_path = f'{scratch_fmriprep}{sub}/func/{sub}_task-movie_run-{run}_desc-confounds_timeseries.tsv'
#         export_path = f'{scratch_fmriprep}derivatives/cleaned/'
#         exportfile = Path(f'{export_path}{sub}_run{run}_clean_space-fsLR_den-91k_bold.dtseries.nii')
#         if not exportfile.is_file(): # if it doesn't exist already, run it
#             if Path(brainpath).is_file():
#                 print(f'running {sub}')
#                 #load confounds
#                 data = pd.read_csv(confound_path, sep='\t')
#                 confounds=data[['global_signal', 'csf', 'white_matter', 'framewise_displacement', 'a_comp_cor_00', 'a_comp_cor_01', 'a_comp_cor_02', 'a_comp_cor_03', 'a_comp_cor_04', 'a_comp_cor_05', 'rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']].to_numpy()

#                 #confounds=data[['global_signal', 'csf', 'white_matter','rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']].to_numpy()
#                 #load brain data
#                 brainimg = load_img(brainpath) # load func img
#                 func_data=np.array(brainimg.dataobj)
#                 func_data = var_to_nan(func_data,0.5)
#                 #clean
#                 #func_cln = clean_img(func,detrend=True,confounds=confounds,low_pass=0.1,t_r=1.5)
#                 func_data_clean = clean(func_data,detrend=True,standardize='zscore',confounds=confounds,low_pass=0.1,t_r=1)
#                 func_cln = nib.Cifti2Image(func_data_clean, brainimg.header)
#                 #export
#                 func_cln.to_filename(f'{export_path}{sub}_run{run}_clean_space-fsLR_den-91k_bold.dtseries.nii')
#             else:
#                 print('dne',sub)
#         else:
#             print(f'skipping {sub}, already ran')

## do the same for the RU HBN data TR=800ms

In [17]:
import numpy as np
from scipy import signal
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
sns.set("paper", "white")

from nilearn.image import load_img
from nilearn.image import clean_img
from nilearn.image import smooth_img
from nilearn.image import index_img
from nilearn.signal import clean
from os import walk, mkdir

from utility import var_to_nan
from pathlib import Path


import nibabel as nib
from nibabel import cifti2
%matplotlib inline

In [5]:
brainpath='/om4/group/gablab/data/HBN/derivatives/fmriprep/sub-NDARZZ740MLM/func/sub-NDARZZ740MLM_task-movieTP_space-fsLR_den-91k_bold.dtseries.nii'
brainimg = load_img(brainpath) # load func img

confound_path='/om4/group/gablab/data/HBN/derivatives/fmriprep/sub-NDARZZ740MLM/func/sub-NDARZZ740MLM_task-movieTP_desc-confounds_timeseries.tsv'

data = pd.read_csv(confound_path, sep='\t')
#confounds=data[['global_signal', 'csf', 'white_matter','rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']].to_numpy()
confounds=data[['global_signal', 'csf', 'white_matter', 'framewise_displacement', 'a_comp_cor_00', 'a_comp_cor_01', 'a_comp_cor_02', 'a_comp_cor_03', 'a_comp_cor_04', 'a_comp_cor_05', 'rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']].to_numpy()
confounds = np.nan_to_num(confounds)

func_data=np.array(brainimg.dataobj)
func_data = var_to_nan(func_data,0.5)

#func_cln = clean_img(func,detrend=True,confounds=confounds,low_pass=0.1,t_r=1.5)
func_data_clean = clean(func_data,detrend=True,standardize='zscore',confounds=confounds,low_pass=0.1,t_r=0.8)
func_cln = nib.Cifti2Image(func_data_clean, brainimg.header)

In [6]:
func_cln.shape

(250, 91282)

In [14]:
# list all budapest subjects
fmriprep = '/om4/group/gablab/data/HBN/derivatives/fmriprep/'
subject_list = list(walk(fmriprep))[0][1]
subject_list.remove('logs')
subject_list.remove('derivatives')

ValueError: list.remove(x): x not in list

In [16]:
#export subject list to a text file
textfile = open("hbn_ru_subjects.txt", "w")
for element in subject_list:
    textfile.write(element + "\n")
textfile.close()

In [19]:
with open("hbn_ru_subjects.txt") as f:
    subject_list = f.read().splitlines()

In [ ]:
subject_list

In [18]:
for sub in subject_list:
    for task in ['TP', 'DM']:
        brainpath=f'/om4/group/gablab/data/HBN/derivatives/fmriprep/{sub}/func/{sub}_task-movie{task}_space-fsLR_den-91k_bold.dtseries.nii'
        confound_path=f'/om4/group/gablab/data/HBN/derivatives/fmriprep/{sub}/func/{sub}_task-movie{task}_desc-confounds_timeseries.tsv'
        export_path = f'/nobackup/scratch/Mon/jsmentch/hbn_cifti_cleaned/{sub}'
        exportfile = Path(f'{export_path}/{sub}_clean_task-movie{task}_space-fsLR_den-91k_bold.dtseries.nii')
        if not exportfile.is_file(): # if it doesn't exist already, run it
            if Path(brainpath).is_file():
                print(f'running {sub}')
                try:
                    mkdir(export_path)
                except OSError as error:
                    print(error)    
                #load confounds
                data = pd.read_csv(confound_path, sep='\t')
                confounds=data[['global_signal', 'csf', 'white_matter', 'framewise_displacement', 'a_comp_cor_00', 'a_comp_cor_01', 'a_comp_cor_02', 'a_comp_cor_03', 'a_comp_cor_04', 'a_comp_cor_05', 'rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']].to_numpy()
                confounds = np.nan_to_num(confounds)
                #confounds=data[['global_signal', 'csf', 'white_matter','rot_x','rot_y','rot_z','trans_x','trans_y','trans_z']].to_numpy()
                #load brain data
                brainimg = load_img(brainpath) # load func img
                func_data=np.array(brainimg.dataobj)
                func_data = var_to_nan(func_data,0.5)
                #clean
                #func_cln = clean_img(func,detrend=True,confounds=confounds,low_pass=0.1,t_r=1.5)
                func_data_clean = clean(func_data,detrend=True,standardize='zscore',confounds=confounds,low_pass=0.1,t_r=0.8)
                func_cln = nib.Cifti2Image(func_data_clean, brainimg.header)
                #export
                func_cln.to_filename(exportfile)
            else:
                print('dne',sub)
        else:
            print(f'skipping {sub}, already ran')

running sub-NDARAC495TJ2
running sub-NDARAC495TJ2
[Errno 17] File exists: '/nobackup/scratch/Mon/jsmentch/hbn_cifti_cleaned/sub-NDARAC495TJ2'
running sub-NDARBM490LK7


KeyboardInterrupt: 